In [4]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
!nvidia-smi

Tue Dec 01 09:50:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 442.50       Driver Version: 442.50       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro P2000       WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   42C    P8    N/A /  N/A |    696MiB /  4096MiB |     21%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
import torch.nn as nn
import torch
import torch.optim as optim

class Net(nn.Module):
    def __init__(self, conv_kernel_size_list, rnn_hidden_size, kernel_mode):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=conv_kernel_size_list[0])
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=conv_kernel_size_list[1])
        self.conv3 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=conv_kernel_size_list[2])
        self.batchnorm = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.drop1 = nn.Dropout(p=0.5)
        self.gru = nn.GRU(input_size=32, hidden_size=rnn_hidden_size, num_layers=2, batch_first=True, dropout=0.5)
        self.drop2 = nn.Dropout(p=0.5)
        self.fc = nn.Linear(rnn_hidden_size, 1)
        self.kernel_mode = kernel_mode
    
    def forward(self, x):
        """
        x: (batch, 6, seq)
        """
        
        if self.kernel_mode == "multi-kernel":
            h0 = torch.cat((self.conv1(x), self.conv2(x), self.conv3(x)), -1) # (batch, 32, seq  - (kernel size - 1))
        else:
            h0 = self.conv1(x)
        h0 = self.batchnorm(h0)
        h0 = self.relu(h0)
        h0 = self.pool(h0)
        # Input (batch, seq, feature)
        h0 = self.drop1(h0)
        o, _ = self.gru(h0.transpose(1, 2))
        # o (batch, seq  - (kernel size - 1), rnn_hidden_size)
        
        score = self.fc(self.drop2(o[:, -1]))
        # self.sigmoid(score)
        # (batch,)
        return score.squeeze(1)

In [6]:
import numpy as np
import json
import torch
from torch.utils import data

class BuffaloKeyStroke(data.Dataset):
    def __init__(self, data_json, keymap_json, sequence_len=100, mean=None, std=None, u_id=1):
        with open(data_json, 'r') as f:
            data = json.load(f)
        with open(keymap_json, 'r') as f:
            self.keyname_to_id = json.load(f)
        
        if mean is None and std is None:
            data_np = []
            for squence, _ in data:
                x = list(map(lambda step: (self.keyname_to_id[step[0]], self.keyname_to_id[step[1]], step[2], step[3], step[4], step[5]), squence))
                data_np += x
            data_np = np.array(data_np)
            self.mean = np.mean(data_np, axis=0)
            self.std = np.std(data_np, axis=0)
        else:
            self.mean = mean
            self.std = std
    
        self.features = []
        self.labels = []
        for sequence, uid in data:
            for i, _ in enumerate(sequence[:-sequence_len]):
                self.features.append(sequence[i:i+sequence_len])
                self.labels.append(int(uid == u_id))
        self.num_positive = (np.array(self.labels) == 1).sum()
        self.num_negative = (np.array(self.labels) == 0).sum()
        self.weights = [self.num_positive if label == 0 else self.num_negative for label in self.labels]

        
    def __getitem__(self, index):
        
        x = self.features[index]
        x = list(map(lambda step: (self.keyname_to_id[step[0]], self.keyname_to_id[step[1]], step[2], step[3], step[4], step[5]), x))
        x = (x - self.mean) / self.std
        
        return torch.FloatTensor(x).transpose(0, 1), self.labels[index]
    
    def __len__(self):
        return len(self.features)

In [9]:
file_path = "./s01_train_s2_test/baseline_train.json"
file_path1 = "./s01_train_s2_test/keyname_to_id.json"

dataset = BuffaloKeyStroke(file_path, file_path1)
x, y = dataset[200]
print(dataset.num_positive)
print(dataset.num_negative)
print(x[0])

5636
258245
tensor([-0.3510,  0.2855,  1.0281,  1.0281, -0.1388,  0.2855, -0.7754, -1.0936,
        -0.9875, -0.8814, -0.7754, -0.6693,  1.0281,  1.0281,  1.0281,  1.0281,
         1.0281,  2.3011,  2.0889, -0.7754, -1.0936, -0.9875, -0.8814, -0.7754,
        -0.6693, -0.1388, -0.8814,  0.0733,  1.2403, -0.7754, -0.3510, -0.6693,
        -0.7754, -0.2449, -0.8814,  0.1794, -0.8814, -0.5632, -0.2449, -0.0328,
        -0.9875,  0.8159, -0.7754, -1.1997, -1.0936, -0.9875, -0.8814, -0.7754,
        -0.0328, -0.3510,  1.5585, -0.2449,  0.1794, -0.8814,  0.1794, -0.7754,
        -1.1997, -0.1388, -0.7754,  0.9220, -0.5632,  0.2855, -1.0936, -0.7754,
        -1.0936, -0.3510, -0.7754,  0.4977,  1.5585, -0.2449,  0.1794,  1.5585,
        -0.8814, -0.7754, -1.1997, -0.3510,  0.4977, -0.8814, -0.2449, -0.5632,
        -1.0936, -0.1388, -0.3510,  0.2855,  0.3916, -0.7754, -1.1997,  0.1794,
         0.6038,  0.1794, -1.0936, -0.8814,  0.7098,  2.3011,  2.0889,  3.1498,
         1.0281,  6.1202,  1

In [10]:
def train(model, train_loader, criterion):
    model.train()
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batch = len(train_loader)
    for i, (batch, labels) in enumerate(train_loader):
        batch = batch.cuda()
        labels = labels.cuda()
        criterion = criterion.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(batch)
        loss = criterion(outputs, labels.type(torch.cuda.FloatTensor))
        loss.backward()
        optimizer.step()
        # print statistics
        epoch_loss += loss.item()
        epoch_acc += outputs.detach().gt(0).long().eq(labels.detach()).float().mean()
    print('Epoch {} Train: loss= {:3f}, Acc= {:3f}'.format(epoch, epoch_loss / num_batch, epoch_acc / num_batch))
    return epoch_loss / num_batch, epoch_acc / num_batch


def test(model, test_loader, criterion):
    model.eval()
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batch = len(test_loader)
    for i, (batch, labels) in enumerate(test_loader):
        batch = batch.cuda()
        labels = labels.cuda()
        criterion = criterion.cuda()

        with torch.no_grad():
            outputs = model(batch)
            loss = criterion(outputs, labels.type(torch.cuda.FloatTensor))
            # print statistics
            epoch_loss += loss.item()
            epoch_acc += outputs.gt(0).long().eq(labels).float().mean()
    print('Epoch {} Test: loss= {:3f}, Acc= {:3f}'.format(epoch, epoch_loss / num_batch, epoch_acc / num_batch))
    return epoch_loss / num_batch, epoch_acc / num_batch

In [34]:
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import BatchSampler
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import os

dataset_path = ["s01_train_s2_test", "s02_train_s1_test", "s12_train_s0_test"]
for p in dataset_path: 
    file_path = "./"+p+"/baseline_train.json"
    file_path1 = "./"+p+"/keyname_to_id.json"
    test_fp = "./baseline_test.json"
    kernel_optional = ["single-kernel", "multi-kernel"]
    for kernel in kernel_optional:
        
        if kernel == "single-kernel":
            ckpt_path_prefix = "./"+p+"/ckpt_s_kernel/ckpt_{}.pth"
        else:
            ckpt_path_prefix = "./"+p+"/ckpt_m_kernel/ckpt_{}.pth"
        
        uid_list = [i for i in range(1,35)]
        
        for uid in uid_list:
            print('UID:', uid)
            model = Net([2,3,4], 8, kernel_mode=kernel).cuda()
            criterion = nn.BCEWithLogitsLoss(reduction='mean').cuda()
            optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
            lr_scheduler = StepLR(optimizer, 120)

            state_dict = {"epoch": 0, "best_model_state": model.state_dict(), "last_model_state": model.state_dict(), "best_acc": 0.0, "train_loss_epoch": [], "train_acc_epoch": [], "test_loss_epoch": [], "test_acc_epoch": []}

            # Load model
            if os.path.exists(ckpt_path_prefix.format(uid)):
                state_dict = torch.load(ckpt_path_prefix.format(uid))
                model.load_state_dict(state_dict['last_model_state'])
                epoch = state_dict['epoch']
                print(epoch, '@#$#@@##$')
            
            train_dataset = BuffaloKeyStroke(file_path, file_path1, u_id=uid)
            train_sampler = WeightedRandomSampler(train_dataset.weights, int(train_dataset.num_positive) * 2, replacement=True)
            train_dataloader = DataLoader(train_dataset, batch_size=64, sampler=train_sampler, drop_last=True)
            
            test_dataset = BuffaloKeyStroke(test_fp, file_path1, 100, train_dataset.mean, train_dataset.std, u_id=uid)
            test_sampler = WeightedRandomSampler(test_dataset.weights, int(test_dataset.num_positive) * 2, replacement=True)
            test_dataloader = DataLoader(test_dataset, batch_size=64, sampler=test_sampler, drop_last=True)    
            
            best_test_loss = float('inf')
            best_test_acc = 0.0
            print('train_path:', p, ', kernel_mode:', kernel)
            
            for epoch in range(state_dict['epoch']+1, 70):  # loop over the dataset multiple times

                train_loss, train_acc = train(model, train_dataloader, criterion)
                test_loss, test_acc = test(model, test_dataloader, criterion)

                state_dict['train_loss_epoch'].append(train_loss)
                state_dict['train_acc_epoch'].append(train_acc)
                state_dict['test_loss_epoch'].append(test_loss)
                state_dict['test_acc_epoch'].append(test_acc)

                lr_scheduler.step()

                # Save model
                if state_dict['best_acc'] < test_acc:
                    state_dict['best_acc'] = test_acc
                    state_dict['best_model_state'] = model.state_dict()

                state_dict['last_model_state'] = model.state_dict()
                state_dict['epoch'] = epoch
                torch.save(state_dict, ckpt_path_prefix.format(uid))

            print('Finished Training')

UID: 36
14 @#$#@@##$
train_path: s01_train_s2_test , kernel_mode: single-kernel
Epoch 15 Train: loss= 0.397811, Acc= 0.842758
Epoch 15 Test: loss= 1.425423, Acc= 0.497869
Epoch 16 Train: loss= 0.379294, Acc= 0.855407
Epoch 16 Test: loss= 1.560041, Acc= 0.494318
Epoch 17 Train: loss= 0.369789, Acc= 0.857019


KeyboardInterrupt: 

In [25]:
uid_list = [i for i in range(1,36)]
acc_avg, acc_sum = 0, 0
for i in uid_list:
    state_dict = torch.load('./s01_train_s2_test/ckpt_s_kernel/ckpt_{}.pth'.format(i))
    print("Uid:", i, "Best_acc", float(state_dict['best_acc']))
    acc_sum += float(state_dict['best_acc'])
print('acc avg:', acc_sum/35)

Uid: 1 Best_acc 0.5985440611839294
Uid: 2 Best_acc 0.7599431872367859
Uid: 3 Best_acc 0.8614130616188049
Uid: 4 Best_acc 0.5666667222976685
Uid: 5 Best_acc 0.9164062738418579
Uid: 6 Best_acc 0.9359681606292725
Uid: 7 Best_acc 0.8417552709579468
Uid: 8 Best_acc 0.7009804248809814
Uid: 9 Best_acc 0.8416510820388794
Uid: 10 Best_acc 0.4964887797832489
Uid: 11 Best_acc 0.9076923131942749
Uid: 12 Best_acc 0.8274038434028625
Uid: 13 Best_acc 0.7776268124580383
Uid: 14 Best_acc 0.975900411605835
Uid: 15 Best_acc 0.9183728694915771
Uid: 16 Best_acc 0.5705819129943848
Uid: 17 Best_acc 0.7042545080184937
Uid: 18 Best_acc 0.9591346383094788
Uid: 19 Best_acc 0.970552921295166
Uid: 20 Best_acc 0.9055989980697632
Uid: 21 Best_acc 0.9580792188644409
Uid: 22 Best_acc 0.9597039818763733
Uid: 23 Best_acc 0.880126953125
Uid: 24 Best_acc 0.7212010025978088
Uid: 25 Best_acc 0.9002840518951416
Uid: 26 Best_acc 0.6306818127632141
Uid: 27 Best_acc 0.875744104385376
Uid: 28 Best_acc 0.7762277126312256
Uid: 29 

In [29]:
uid_list = [i for i in range(1,36)]
acc_avg, acc_sum = 0, 0
for i in uid_list:
    state_dict = torch.load('./s01_train_s2_test/ckpt_m_kernel/ckpt_{}.pth'.format(i))
    print("Uid:", i, "Best_acc", state_dict['best_acc'])
    acc_sum += float(state_dict['best_acc'])    
print('acc avg:', acc_sum/35)

Uid: 1 Best_acc tensor(0.6618, device='cuda:0')
Uid: 2 Best_acc tensor(0.7028, device='cuda:0')
Uid: 3 Best_acc tensor(0.9423, device='cuda:0')
Uid: 4 Best_acc tensor(0.6169, device='cuda:0')
Uid: 5 Best_acc tensor(0.9333, device='cuda:0')
Uid: 6 Best_acc tensor(0.9400, device='cuda:0')
Uid: 7 Best_acc tensor(0.8916, device='cuda:0')
Uid: 8 Best_acc tensor(0.7273, device='cuda:0')
Uid: 9 Best_acc tensor(0.7882, device='cuda:0')
Uid: 10 Best_acc tensor(0.5730, device='cuda:0')
Uid: 11 Best_acc tensor(0.9502, device='cuda:0')
Uid: 12 Best_acc tensor(0.8793, device='cuda:0')
Uid: 13 Best_acc tensor(0.7298, device='cuda:0')
Uid: 14 Best_acc tensor(0.9947, device='cuda:0')
Uid: 15 Best_acc tensor(0.9135, device='cuda:0')
Uid: 16 Best_acc tensor(0.5374, device='cuda:0')
Uid: 17 Best_acc tensor(0.7026, device='cuda:0')
Uid: 18 Best_acc tensor(0.9498, device='cuda:0')
Uid: 19 Best_acc tensor(0.9246, device='cuda:0')
Uid: 20 Best_acc tensor(0.9131, device='cuda:0')
Uid: 21 Best_acc tensor(0.997

In [45]:
x = torch.randn(2, 3)
print(x)
print('================')
a = torch.cat((x, x, x), 0)
print(a , a.shape)
print('================')
b = torch.cat((x, x, x), -1)
print(b, b.shape)

y = torch.randn(6, 2, 8)
z = torch.randn(6, 2, 12)
t = torch.randn(6, 2, 12)
yz = torch.cat((y, z, t), -1)
print(yz.shape)


tensor([[ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940]])
tensor([[ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940],
        [ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940],
        [ 0.8913, -1.1462, -0.4117],
        [ 0.8636, -0.2382, -0.5940]]) torch.Size([6, 3])
tensor([[ 0.8913, -1.1462, -0.4117,  0.8913, -1.1462, -0.4117,  0.8913, -1.1462,
         -0.4117],
        [ 0.8636, -0.2382, -0.5940,  0.8636, -0.2382, -0.5940,  0.8636, -0.2382,
         -0.5940]]) torch.Size([2, 9])
torch.Size([6, 2, 32])
